In [112]:
import csv
import spacy


nlp = spacy.load('es_core_news_lg')

In [113]:
import itertools
def check_len(line):
    if len(line.split()) < 3:
        line = line.split()
        line.append('Not.Tagged')
        line = ' '.join(line)
        return line
    else:
        return line

def clean_text(line):
    line = line.lower()
    line = line.replace('&eacute;', 'é')
    line = line.replace('&aacute;', 'á')
    line = line.replace('&iacute;', 'í')
    line = line.replace('&oacute;', 'ó')
    line = line.replace('&uacute;', 'ú')
    line = line.replace('&ntilde;', 'ñ')
    line = line.replace('&acirc;', 'â')
    line = line.replace('&iexcl;', ' ¡ ')
    line = line.replace('&iquest;', ' ¿ ')
    line = line.replace('&quot', '\"')
    return line.split()

count = 0
with open('C:\\Users\\Jerem\\Downloads\\cf-output.txt', encoding= 'utf8') as corpus:
    all_sents = []
    all_tags = []
    current_sentence = []
    current_tags = []
    for line in corpus:
        line = check_len(line)
        clean = clean_text(line)
        word = clean[0]
        pos = clean[2]
        if pos.lower() != 'punct.final':
            current_sentence.append(word)
            current_tags.append(pos)
        elif pos.lower() == 'punct.final' and word in ['.', '!', '?', ]:
            all_sents.append(current_sentence)
            all_tags.append(current_tags)
            current_sentence =[]
            current_tags = []



punct = ['!', ':', '.', ',', ';', '=', '-', ')', '(', '?', '+', '¡', '¿', '\"', '\":', '\";']


with open('C:\\Users\\Jerem\\Desktop\\counterfactuals.csv', encoding='utf8', mode='a', newline='') as out:
    writer = csv.writer(out)
    for sent, tags in zip(all_sents, all_tags):
        if len(sent) == len(tags):
            filtered_sent = []
            filtered_tags = []
            for ind, word in enumerate(sent):
                if word not in punct:
                    filtered_sent.append(word)
                    filtered_tags.append(tags[ind])
        temp_sent = ' '.join(filtered_sent)
        temp_tags = ' '.join(filtered_tags)
        cf = []
        for tag in filtered_tags:
            if 'cf' in tag:
                cf.append(tag)
        doc = nlp(temp_sent)
        verb_moods = [token.morph.get("Mood") for token in doc if token.pos_ == 'VERB']
        verb_moods = [mood for subls in verb_moods for mood in subls if isinstance(subls, list)]
        mood_counts = {}
        for mood in set(verb_moods):
            mood_counts[mood] = verb_moods.count(mood)
        if len(cf) > 0:
            writer.writerow([temp_sent, mood_counts, temp_tags, str(list(set(cf))), 1])
        else:
            writer.writerow([temp_sent, mood_counts, temp_tags, 'None', 0])

